# Blockchain Document Loader

## Overview

The intention of this notebook is to provide a means of testing functionality in the Langchain Document Loader for Blockchain.

Initially this Loader supports:


*   Ethereum Maninnet, Ethereum Testnet, Polgyon Mainnet, Polygon Testnet (default is eth-mainnet)
*   Alchemy's getNFTsForCollection API

It can be extended if the community finds value in this loader.  Specifically:

*   Additional APIs can be added (e.g. Tranction-related APIs)

To run this notebook, the user will need:


*   An OpenAI key (for OpenAI models)
*   A free [Alchemy API Key](https://https://www.alchemy.com/)





## Setup

In [48]:
%pip install langchain -q

Note: you may need to restart the kernel to use updated packages.


In [49]:
from langchain.document_loaders import BlockchainDocumentLoader
from langchain.document_loaders.blockchain import BlockchainType
import os

In [50]:
alchemyApiKey = "get your own key from https://www.alchemy.com/"
os.environ["ALCHEMY_API_KEY"] = alchemyApiKey

## Create a Blockchain Document Loader

### Option 1: Ethereum Mainnet (default BlockchainType)

In [24]:
contractAddress = "0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d"  # Bored Ape Yacht Club contract address

blockchainType = BlockchainType.ETH_MAINNET  # default value, optional parameter

blockchainLoader = BlockchainDocumentLoader(contractAddress)

nfts = blockchainLoader.load()

nfts[:2]

[Document(page_content="{'contract': {'address': '0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d'}, 'id': {'tokenId': '0x0000000000000000000000000000000000000000000000000000000000000000', 'tokenMetadata': {'tokenType': 'ERC721'}}, 'title': '', 'description': '', 'tokenUri': {'gateway': 'https://alchemy.mypinata.cloud/ipfs/QmeSjSinHpPnmXmspMjwiXyN6zS4E9zccariGR3jxcaWtq/0', 'raw': 'ipfs://QmeSjSinHpPnmXmspMjwiXyN6zS4E9zccariGR3jxcaWtq/0'}, 'media': [{'gateway': 'https://nft-cdn.alchemy.com/eth-mainnet/415d618f5fef7bfe683e02d4653c4289', 'thumbnail': 'https://res.cloudinary.com/alchemyapi/image/upload/thumbnailv2/eth-mainnet/415d618f5fef7bfe683e02d4653c4289', 'raw': 'ipfs://QmRRPWG96cmgTn2qSzjwr2qvfNEuhunv6FNeMFGa9bx6mQ', 'format': 'png', 'bytes': 133270}], 'metadata': {'image': 'ipfs://QmRRPWG96cmgTn2qSzjwr2qvfNEuhunv6FNeMFGa9bx6mQ', 'attributes': [{'value': 'Silver Hoop', 'trait_type': 'Earring'}, {'value': 'Orange', 'trait_type': 'Background'}, {'value': 'Robot', 'trait_type': 'Fur'}, {'val

### Option 2: Polygon Mainnet

In [36]:
contractAddress = (
    "0x448676ffCd0aDf2D85C1f0565e8dde6924A9A7D9"  # Polygon Mainnet contract address
)

blockchainType = BlockchainType.POLYGON_MAINNET

blockchainLoader = BlockchainDocumentLoader(contractAddress, blockchainType)

nfts = blockchainLoader.load()

nfts[:2]

[Document(page_content="{'contract': {'address': '0x448676ffcd0adf2d85c1f0565e8dde6924a9a7d9'}, 'id': {'tokenId': '0x01', 'tokenMetadata': {'tokenType': 'ERC1155'}}, 'title': 'Wyatt Horton #0001', 'description': 'A sleepy capybara', 'tokenUri': {'gateway': 'https://storage.googleapis.com/minted-nfts/smoothstack/avatars/metadata/1.json', 'raw': 'https://storage.googleapis.com/minted-nfts/smoothstack/avatars/metadata/1.json'}, 'media': [{'gateway': 'https://nft-cdn.alchemy.com/matic-mainnet/9085e06ff9f6c9074de91801d1c72d26', 'thumbnail': 'https://res.cloudinary.com/alchemyapi/image/upload/thumbnailv2/matic-mainnet/9085e06ff9f6c9074de91801d1c72d26', 'raw': 'https://storage.googleapis.com/minted-nfts/smoothstack/avatars/images/1.png', 'format': 'png', 'bytes': 769622}], 'metadata': {'name': 'Wyatt Horton #0001', 'description': 'A sleepy capybara', 'image': 'https://storage.googleapis.com/minted-nfts/smoothstack/avatars/images/1.png', 'attributes': [{'value': 'Avatar', 'trait_type': 'Type'}

## (Optional) Using the Blockchain Document Loader

### Setup Splitter and Index

In [37]:
%pip install sentence_transformers chromadb openai tiktoken -q

Note: you may need to restart the kernel to use updated packages.


In [38]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [39]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=0)

docs = text_splitter.split_documents(nfts)
print("NUMBER OF DOCUMENTS: ", len(docs))

NUMBER OF DOCUMENTS:  424


In [40]:
index = VectorstoreIndexCreator(
    embedding=HuggingFaceEmbeddings(), text_splitter=text_splitter
).from_loaders([blockchainLoader])

Using embedded DuckDB without persistence: data will be transient


## Setup Models and Chains

In [42]:
openAiKey = "put OpenAI key here"
os.environ["OPENAI_API_KEY"] = openAiKey

In [31]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

### Retrieval Chain

In [43]:
llmOpenAI = OpenAI()

chainQA = RetrievalQA.from_chain_type(
    llm=llmOpenAI,
    chain_type="map_reduce",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    input_key="question",
)

In [44]:
chainQA.run("What are some of the popular attributes?")



> Entering new RetrievalQA chain...

> Finished chain.


' Popular attributes include "Avatar" (Type), "Character" (Category), and "Human" or "Wizard" (Class).'

In [ ]:
chainQA.run("How many NFTs are there?")



> Entering new RetrievalQA chain...

> Finished chain.


' There are 10,000 unique Bored Ape NFTs.'